In [21]:
import pandas as pd
import numpy as np

import requests
from urllib.parse import urlunparse
from bs4 import BeautifulSoup

from dateutil import parser
import matplotlib.pyplot as plt

In [22]:
urls = ['https://www.odmp.org/search?from=2005&to=2022&filter=k9&o=']

for i in range(1, 13):
    urls.append('https://www.odmp.org/search?from=2005&to=2022&filter=k9&o={}'.format(i*25))

In [23]:
fullOfficerData = {
        'Name': [],
        'PD': [],
        'EOW': [],
        'Cause': [],
    }

fullOfficerDF = pd.DataFrame(fullOfficerData)


In [24]:
fullOfficerData = {
        'Name': [],
        'PD': [],
        'EOW': [],
        'Cause': [],
    }

fullOfficerDF = pd.DataFrame(fullOfficerData)

for i in range(len(urls)):
    response = requests.get(urls[i])

    html_string = response.text
    document = BeautifulSoup(html_string, "html.parser")

    temporaryPageOfficerData = {
        'Name': [],
        'PD': [],
        'EOW': [],
        'Cause': [],
    }

    officerDetails = document.find_all("div", attrs={"class": "officer-details"})

    for officer in officerDetails:
        temporaryPageOfficerData['Name'].append(officer.find("a").text)
        temporaryPageOfficerData['PD'].append(officer.find("p").text)
        temporaryPageOfficerData['EOW'].append(officer.find_all("p")[1].text)
        temporaryPageOfficerData['Cause'].append(officer.find_all("p")[2].text)
        
    officerDF = pd.DataFrame(temporaryPageOfficerData) 
    fullOfficerDF = pd.concat([fullOfficerDF, officerDF])

In [25]:
fullOfficerDF

,Name,PD,EOW,Cause
0,K9 Mina,United States Department of Homeland Security ...,"EOW: Monday, May 2, 2022",Cause of Death: Animal related
1,K9 Jinx,"El Paso County Sheriff's Office, CO","EOW: Monday, April 11, 2022",Cause of Death: Gunfire
2,K9 Major,"Franklin County Sheriff's Office, NC","EOW: Sunday, April 10, 2022",Cause of Death: Gunfire
3,K9 Dash,"Shepherdsville Police Department, KY","EOW: Wednesday, March 9, 2022",Cause of Death: Gunfire
4,K9 Ciro,"Humphreys County Sheriff's Office, TN","EOW: Thursday, March 3, 2022",Cause of Death: Fire
...,...,...,...,...
16,K9 R.J.,"Phoenix Police Department, AZ","EOW: Saturday, August 13, 2005",Cause of Death: Vehicular assault
17,K9 Pasco,"Pleasanton Police Department, CA","EOW: Tuesday, July 19, 2005",Cause of Death: Fall
18,K9 Arco,"Marion County Sheriff's Office, IN","EOW: Monday, July 11, 2005",Cause of Death: Gunfire
19,K9 Gunner,"Prince William County Police Department, VA","EOW: Monday, June 6, 2005",Cause of Death: Gunfire (Inadvertent)


In [26]:
fullOfficerDF.reset_index(inplace=True)
fullOfficerDF.drop('index', axis=1, inplace=True)
fullOfficerDF

,Name,PD,EOW,Cause
0,K9 Mina,United States Department of Homeland Security ...,"EOW: Monday, May 2, 2022",Cause of Death: Animal related
1,K9 Jinx,"El Paso County Sheriff's Office, CO","EOW: Monday, April 11, 2022",Cause of Death: Gunfire
2,K9 Major,"Franklin County Sheriff's Office, NC","EOW: Sunday, April 10, 2022",Cause of Death: Gunfire
3,K9 Dash,"Shepherdsville Police Department, KY","EOW: Wednesday, March 9, 2022",Cause of Death: Gunfire
4,K9 Ciro,"Humphreys County Sheriff's Office, TN","EOW: Thursday, March 3, 2022",Cause of Death: Fire
...,...,...,...,...
316,K9 R.J.,"Phoenix Police Department, AZ","EOW: Saturday, August 13, 2005",Cause of Death: Vehicular assault
317,K9 Pasco,"Pleasanton Police Department, CA","EOW: Tuesday, July 19, 2005",Cause of Death: Fall
318,K9 Arco,"Marion County Sheriff's Office, IN","EOW: Monday, July 11, 2005",Cause of Death: Gunfire
319,K9 Gunner,"Prince William County Police Department, VA","EOW: Monday, June 6, 2005",Cause of Death: Gunfire (Inadvertent)


In [27]:
#now let's clean up the data
for i in range(len(fullOfficerDF['Name'])):
    #First let's cut "K9" off all the dog names
    fullOfficerDF.at[i, 'Name'] = fullOfficerDF.at[i, 'Name'][3:]
    #Now let's grab only the last two characters of PD to identify the state
    fullOfficerDF.at[i, 'PD'] = fullOfficerDF.at[i, 'PD'][-2:]
    #Let's take the EOW and cut off the day of the week and convert the date to a datetime object
    fullOfficerDF.at[i, 'EOW'] = fullOfficerDF.at[i, 'EOW'].partition(", ")[2]
    fullOfficerDF.at[i, 'EOW'] = parser.parse(fullOfficerDF.at[i, 'EOW'])
    #Let's remove the "Cause of Death:" string from the Cause column
    fullOfficerDF.at[i, 'Cause'] = fullOfficerDF.at[i, 'Cause'].partition(": ")[2]
    
fullOfficerDF

,Name,PD,EOW,Cause
0,Mina,US,2022-05-02 00:00:00,Animal related
1,Jinx,CO,2022-04-11 00:00:00,Gunfire
2,Major,NC,2022-04-10 00:00:00,Gunfire
3,Dash,KY,2022-03-09 00:00:00,Gunfire
4,Ciro,TN,2022-03-03 00:00:00,Fire
...,...,...,...,...
316,R.J.,AZ,2005-08-13 00:00:00,Vehicular assault
317,Pasco,CA,2005-07-19 00:00:00,Fall
318,Arco,IN,2005-07-11 00:00:00,Gunfire
319,Gunner,VA,2005-06-06 00:00:00,Gunfire (Inadvertent)


In [28]:
print(fullOfficerDF)

       Name  PD                  EOW                  Cause
0      Mina  US  2022-05-02 00:00:00         Animal related
1      Jinx  CO  2022-04-11 00:00:00                Gunfire
2     Major  NC  2022-04-10 00:00:00                Gunfire
3      Dash  KY  2022-03-09 00:00:00                Gunfire
4      Ciro  TN  2022-03-03 00:00:00                   Fire
..      ...  ..                  ...                    ...
316    R.J.  AZ  2005-08-13 00:00:00      Vehicular assault
317   Pasco  CA  2005-07-19 00:00:00                   Fall
318    Arco  IN  2005-07-11 00:00:00                Gunfire
319  Gunner  VA  2005-06-06 00:00:00  Gunfire (Inadvertent)
320    Yari  TR  2005-01-01 00:00:00             Heatstroke

[321 rows x 4 columns]
